### Machine Learning Major Research Project

Researcher: Alistair Clarke

Season: Fall 2018

In this research project, we will look at an ML program which classifies fruit images and determines the correct type or class of fruit the image belongs too.

Project Scope:

- Image Data Augmentation
- Different Models (KNN and Neural Networks)
- Features (Principal Components Analysis - PCAs)

Project Variables

- Kaggle's Fruit365 Dataset
- Google Downloaded Fruit Dataset
- Augmented Fruit Dataset

Number and Sequence of Iterations

1. Google Downloaded Fruit Dataset
2. Google Downloaded Fruit Dataset with Kaggle Fruit365 Dataset
3. Google Downloaded Fruit Dataset with Augmented Google Downloaded Fruit Dataset
4. Google Downloaded Fruit Dataset with Augmented Google Downloaded Fruit Dataset and Kaggle Fruit365 Dataset
5. Google Downloaded Fruit Dataset with Individual Augmented Google Downloaded Fruit Dataset (x6).

Research Questions

1. Is Augmentation Effective?
2. Which Model performs the best?
3. Which Augmentation Type is best?
4. Why were the K values for KNN chosen?
5. Why were the models (i.e. KNN, Neural Network) chosen?
6. Why were PCAs used for feature extraction?

### Importation of Libraries

In [ ]:
# Import skimage, scipy, pandas, numpy, matplotlib, sklearn libraries

import os
import skimage
import numpy as np
import pandas as pd
from skimage.color import rgb2gray
from skimage import data
from skimage import exposure
from skimage import io
from scipy import ndimage

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from preamble import *
plt.rcParams['image.cmap'] = "gray"

### Feature Extraction Function

- Calculates the color mean/average of each pixel
- Stores the average color pixel values in a list

In [ ]:
# Function which gets the raw data

def load_raw_data(path):
    
    fruits = []
    
    for root, dirs, files in os.walk(path):
        for file_ in files:
            img_file = os.path.join(root, file_)
                    
            image = io.imread(img_file)  # Read image file

            img_gray = rgb2gray(image)   # Convert image to grayscale

            g_values = img_gray < 0.9    # Generate a boolean array of values < 0.9

            pixel_avg = image[g_values]  # Calculate the avg true pixel values

            pixels = pixel_avg           # RGB pixel values

            rgb_avgs = np.mean(pixels, axis=(0))   # Calculate the avg pixel value from all the columns (axis=0)

            path = os.path.dirname(img_file)   # Combine img and path to create entire directory name

            label = os.path.basename(path)     # Extract out the folder name to use as the label
            
            f_label = label.replace(" ", "")   # Remove white spaces from labels

            img_array = np.append(rgb_avgs, f_label)   # Store avgs in numpy array
            
            fruits.append(img_array)  # Append color avgs to fruit list

    return fruits

### Save Generated Features as CSV Function

- Saves the numpy array of features as a csv file

In [ ]:
# Save Dataset as CSV function

def save_dataset(contents, file):

    all_fruits = np.array(contents)  # Convert contents to a numpy array

    df = pd.DataFrame(all_fruits)  # Save vector of features as a csv using Pandas library

    df.to_csv(file)

### Download Dataset Function

- Downloads the save csv file of features 
- Removes unwanted columns

In [ ]:
# Function to load in CSV dataset

def download_dataset(path, col_labels):
    
    features = pd.read_csv(path)   # Import CSV file of features

    f_vector = np.array(features)  # Convert features to numpy array
        
    fruit_features = pd.DataFrame(data=f_vector, columns=col_labels)  # Convert feature vector to Dataframe

    fruit_features.columns   # Get all columns 

    fruit_features.drop('Index', axis=1, inplace=True)   # Drop Index column from dataset
    
    return fruit_features

### Data Splitter Function

- Splits the data using a Stratified Shuffle Split into training and testing
- Idea: so that the data would be distributed evenly

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Split Dataset Function
# 70% Training, 15% Testing, 15% Validation

def splitter(feature_vector):
    
    # Stratified Sampling Split (Training and Testing)

    # Splits data once
    # 70% training, 15% testing, 15% validation

    split = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)

    for train_index, test_index in split.split(feature_vector, feature_vector['Label']):  # Set label as the strata
        
        print('Train size: {}, test size: {}'.format(train_index.shape, test_index.shape))

        strat_train_set = feature_vector.loc[train_index]
        strat_test_set = feature_vector.loc[test_index]
        
    return strat_train_set, strat_test_set

### Combine Feature Vectors Functions

- Combines multiple feature vectors for training and testing purposes

In [ ]:
# Add new data to original training set 

def append_new_data(old_data, new_data):
    
    total_fruits = np.concatenate((old_data, new_data), axis = 0)
    
    return total_fruits

In [ ]:
# Combine all three datasets

def append_triple_data(old_data, new_data, newer_data):
    
    two_fruits = np.concatenate((old_data, new_data), axis = 0)
    three_fruits = np.concatenate((two_fruits, newer_data), axis = 0)
    
    return three_fruits

### Prepare The Data Functions

- Removes the labels from the feature vector
- Checks for any incomplete values within the feature vector

In [ ]:
# Prepare the training data for the model

def prepare_data(strat_train_set):    

    fruit = strat_train_set.drop('Label', axis=1)  # Remove the output labels from the training set (Define column using axis=1)

    fruit_labels = strat_train_set['Label'].copy()  # Copy column contents in fruit_labels

    sample_incomplete_rows = fruit[fruit.isnull().any(axis=1)].head()  # Check for rows that have empty or null values
    
    return fruit, fruit_labels, sample_incomplete_rows

In [ ]:
# Prepare the combined training data for the knn model

def prepare_combined_data(strat_train_set, new_strat_train_set):

    fruit = strat_train_set.drop('Label', axis=1)     # Remove the output labels from the training set
    new_fruit = new_strat_train_set.drop('Label', axis=1)

    # Copy column contents in fruit_labels

    fruit_labels = strat_train_set['Label'].copy()
    new_labels = new_strat_train_set['Label'].copy()
    
    # Check for rows that have empty or null values

    sample_incomplete_rows = fruit[fruit.isnull().any(axis=1)].head()

    sample_incomplete_rows_2 = new_fruit[new_fruit.isnull().any(axis=1)].head()
    
    return fruit, fruit_labels, new_fruit, new_labels, sample_incomplete_rows, sample_incomplete_rows_2

In [ ]:
# Prepare the combined training data for the model

def prepare_triple_data(strat_train_set, new_strat_train_set, newer_strat_train_set):
    
    # Remove the output labels from the training set
    # Define column using axis=1

    fruit = strat_train_set.drop('Label', axis=1)
    new_fruit = new_strat_train_set.drop('Label', axis=1)
    newer_fruit = newer_strat_train_set.drop('Label', axis=1)

    # Copy column contents in fruit_labels

    fruit_labels = strat_train_set['Label'].copy()
    new_labels = new_strat_train_set['Label'].copy()
    newer_labels = newer_strat_train_set['Label'].copy()
    
    # Check for rows that have empty or null values

    sample_incomplete_rows = fruit[fruit.isnull().any(axis=1)].head()

    sample_incomplete_rows_2 = new_fruit[new_fruit.isnull().any(axis=1)].head()
    
    sample_incomplete_rows_3 = newer_fruit[newer_fruit.isnull().any(axis=1)].head()
    
    return fruit, fruit_labels, new_fruit, new_labels, newer_fruit, newer_labels, sample_incomplete_rows, sample_incomplete_rows_2, sample_incomplete_rows_3

### KNN Model Training Functions

- Inputs fruit features and labels
- Trains KNN Model where K = 1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Train KNN model 

def train_knn_model(fruit, labels):

    # Splitting the data (Features, Labels)
    # F(x) = y, where X is the feature vectors and y output labels
    # train_test_split return 4 values, which are assigned to each of the four variable defined

    X_train, X_val, y_train, y_val = train_test_split(fruit, labels, test_size=0.15, random_state=42)
    
    knn = KNeighborsClassifier(n_neighbors = 3)    # Implement the KNN model, with 3 neighbors

    knn.fit(X_train, y_train)  # Train partial training set on model (i.e. KNN)
    
    knn_scores = cross_val_score(knn, fruit, labels, cv=10)
    
    return knn, knn_scores, X_val, y_val, X_train, y_train

In [ ]:
# Train Combined Data KNN Model 

def train_combined_model(fruit, labels, new_fruit, new_labels):

    # Splitting the data (Features, Labels)
    # F(x) = y, where X is the feature vectors and y output labels
    # train_test_split return 4 values, which are assigned to each of the four variable defined

    X_train, X_val, y_train, y_val = train_test_split(fruit, labels, test_size=0.15, random_state=42)
    
    x_train, x_test, Y_train, y_test = train_test_split(new_fruit, new_labels, random_state=42)
    
    # Combine training data
    
    X_train = np.concatenate((X_train, x_train), axis = 0)
    y_train = np.concatenate((y_train, Y_train), axis = 0)

    # Implement the KNN model, with 3 neighbors
    
    knn = KNeighborsClassifier(n_neighbors = 3)

    knn.fit(X_train, y_train) # Train partial training set on model (i.e. KNN)
    
    return knn, X_val, y_val, X_train, y_train

In [ ]:
# Train Combined Data KNN Model 

def train_evaluate_model(fruit, labels):

    # Splitting the data (Features, Labels)
    # F(x) = y, where X is the feature vectors and y output labels
    # train_test_split return 4 values, which are assigned to each of the four variable defined

    X_train, X_test, y_train, y_test = train_test_split(fruit, labels, random_state=42)
    
    # Implement the KNN model, with 3 neighbors
    
    knn = KNeighborsClassifier(n_neighbors = 3)
    
    # Train partial training set on model (i.e. KNN)

    knn.fit(X_train, y_train)
    
    return knn, X_train, y_train

In [ ]:
# Train Combined Data KNN Model 

def train_triple_model(fruit, labels, new_fruit, new_labels, newer_fruit, newer_labels):

    # Splitting the data (Features, Labels)
    # F(x) = y, where X is the feature vectors and y output labels
    # train_test_split return 4 values, which are assigned to each of the four variable defined

    X_train, X_val, y_train, y_val = train_test_split(fruit, labels, test_size=0.15, random_state=42)
    
    x_train, x_test, Y_train, y_test = train_test_split(new_fruit, new_labels, random_state=42)
    
    k_x_train, k_x_test, k_Y_train, k_y_test = train_test_split(newer_fruit, newer_labels, random_state=42)
    
    # Combine training data
    
    X_train = np.concatenate((X_train, x_train), axis = 0)
    y_train = np.concatenate((y_train, Y_train), axis = 0)
    
    X_train = np.concatenate((X_train, k_x_train), axis = 0)
    y_train = np.concatenate((y_train, k_Y_train), axis = 0)

    # Implement the KNN model, with 3 neighbors
    
    knn = KNeighborsClassifier(n_neighbors = 3)
    
    # Train partial training set on model (i.e. KNN)

    knn.fit(X_train, y_train)
    
    return knn, X_val, y_val, X_train, y_train

### PCA Feature Generation Functions

- Converts the feature space of the original feature vector to produce more informative features that would increase the overall performance of the KNN Model.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# PCA Feature Extraction Function

def PCA_features(fruit, labels, x_val, x_train):
    
    # StandardScaler normalizes the incoming data rows have a mean of zero, columns std of 1

    scaler = StandardScaler()
    scaler.fit(fruit)
    X_scaled = scaler.transform(fruit)
    
    # Keep the first two principal components of the data
    
    pca = PCA(n_components=2, whiten=True, random_state=42).fit(x_train)
    
    # Apply PCA feature space transformation on both sets (training and test sets)
    
    X_train_pca = pca.transform(x_train)
    X_test_pca = pca.transform(x_val)
    
    pca.fit(X_scaled)  # Train PCA model to fruit data
    
    X_pca = pca.transform(X_scaled)  # Transform data onto the first two principal components

    # Plot the principal components (discrete scattered plot), colored by class labels
    
    plt.figure(figsize=(8, 8))
    mglearn.discrete_scatter(X_pca[:, 0], X_pca[:, 1], labels)
    
    # np.unique - ensures that labels are unique
    
    plt.legend(np.unique(labels), loc="best")
    
    plt.gca().set_aspect("equal")
    plt.xlabel("First principal component")
    plt.ylabel("Second principal component")
    
    return X_train_pca, X_test_pca

### Train KNN Model with PCA Features Function

- Trains the KNN Model using the PCA generated training data

In [ ]:
# PCA Training Function

def pca_train_model(model, x_train_pca, x_test_pca, y_train, y_test):
    
    # Model training

    model.fit(x_train_pca, y_train)

    accura = model.score(x_test_pca, y_test)

    return print("Test set accuracy: {:.2f}".format(accura))

### Google Downloaded Fruit Dataset

Steps to training:

- Load in the data from the respective directory
- Generate color mean features and save those features as a csv
- Read the save data into a dataframe
- Set the column values from the dataframe
- Display the first 5 rows of feature vector
- Split the feature vector into training and test sets
- Make csv copies of splitted data sets for final testing
- Prepare the data from the KNN model
- Train KNN Model
- Generate new features using PCA (First two PC)
- Retrain KNN Model using PCA features
- Output training results

In [ ]:
# Base Directory

base_dir = '../Major Project 2018/Fruit Classifier'

path = base_dir + '/google-data'

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/google_downloaded_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

COL_NAMES = ['Index','Red','Green','Blue','Label']

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

d_new_strat_train, d_new_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

data, labels, incomplete_samples = prepare_data(d_new_strat_train)

# Displays incomplete data

incomplete_samples    

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, knn_score, x_val, y_val, x_train, y_train = train_knn_model(data, labels)

In [ ]:
# Perform PCA on dataset

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_3 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_3, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights, the model prediction results were as follows:

- K = 1 (47% Accuracy)
- K = 3 (40% Accuracy)
- K = 5 (45% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors

From these results it can be concluded that using a value of K which is less 3 or greater than 3 of the total number possible identifiable classes would be the best viable option. 

The ultimate reason why this new dataset accuracy was much lower than the Fruit365 dataset's, was because the new dataset (i.e. Google Downloaded Fruit Dataset) had alot of fruits in different classes with the same colors, and because of that it was difficult for the model to differentiate between the different classes.

To solve this problem adding for informative features (e.g. shape) would improve the performance.

### Google Downloaded Fruit Dataset with Kaggle's Fruit365 Dataset 

Steps to training:

- Load in the data from the respective directory
- Generate color mean features and save those features as a csv
- Read the save data into a dataframe
- Set the column values from the dataframe
- Display the first 5 rows of feature vector
- Split the feature vector into training and test sets
- Make csv copies of splitted data sets for final testing
- Prepare the data from the KNN model
- Train KNN Model
- Generate new features using PCA (First two PC)
- Retrain KNN Model using PCA features
- Output training results

In [ ]:
path = base_dir + "/kaggle-data"

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/kaggle_fruit_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

k_new_strat_train, k_new_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, k_data, k_labels, incomplete_samples, incomplete_samples_2 = prepare_combined_data(d_new_strat_train, k_new_strat_train)

# Display any incomplete data

print(incomplete_samples)
print(incomplete_samples_2)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_combined_model(g_data, g_labels, k_data, k_labels)

In [ ]:
# Perform PCA on dataset

data = append_new_data(g_data, k_data)
labels = append_new_data(g_labels, k_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_4 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_4, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights, the model prediction results were as follows:

- K = 1 (47% Accuracy)
- K = 3 (47% Accuracy)
- K = 5 (45% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors

From these results it can be concluded that using a value of K which is less than or equal to 3 of the total number possible identifiable classes would be the best viable option.

### Google Downloaded Fruit Dataset with Google Downloaded Augmented Dataset 

Steps to training:

- Load in the data from the respective directory
- Generate color mean features and save those features as a csv
- Read the save data into a dataframe
- Set the column values from the dataframe
- Display the first 5 rows of feature vector
- Split the feature vector into training and test sets
- Make csv copies of splitted data sets for final testing
- Prepare the data from the KNN model
- Train KNN Model
- Generate new features using PCA (First two PC)
- Retrain KNN Model using PCA features
- Output training results

In [ ]:
path = base_dir + "/google-augmented-data"

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/google_augmented_fruit_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

g_aug_new_strat_train, g_aug_new_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, g_aug_data, g_aug_labels, incomplete_samples, incomplete_samples_2 = prepare_combined_data(d_new_strat_train, g_aug_new_strat_train)

print(incomplete_samples)

print(incomplete_samples_2)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_combined_model(g_data, g_labels, g_aug_data, g_aug_labels)

In [ ]:
# Perform PCA on dataset

data = append_new_data(g_data, g_aug_data)
labels = append_new_data(g_labels, g_aug_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_5 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_5, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights, the model prediction results were as follows:

- K = 1 (90% Accuracy)
- K = 3 (78% Accuracy)
- K = 5 (68% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors

From these results it can be concluded that using a value of K which is less than or equal to 3 of the total number possible identifiable classes would be the best viable option.

### Google Downloaded Fruit Dataset, Google Downloaded Augmented Dataset and Kaggle Fruit365 Dataset Combined

Steps to training:

- Prepare the data from the KNN model
- Train KNN Model
- Generate new features using PCA (First two PC)
- Retrain KNN Model using PCA features
- Output training results

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, g_aug_data, g_aug_labels, k_data, k_labels, incomplete_1, incomplete_2, incomplete_3 = prepare_triple_data(d_new_strat_train, g_aug_new_strat_train, k_new_strat_train)

print(incomplete_1)
print(incomplete_2)
print(incomplete_3)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_triple_model(g_data, g_labels, g_aug_data, g_aug_labels, k_data, k_labels)

In [ ]:
# Perform PCA on dataset

data = append_triple_data(g_data, g_aug_data, k_data)
labels = append_triple_data(g_labels, g_aug_labels, k_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_6 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_6, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights, the model prediction results were as follows:

- K = 1 (88% Accuracy)
- K = 3 (80% Accuracy)
- K = 5 (68% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors

From these results it can be concluded that using a value of K which is less than or equal to 3 of the total number possible identifiable classes would be the best viable option.

### Evaluating on Testset

- Evaluate on test set using best model
- Train on entire training set and evaluate on test set


In [ ]:
CSV_FILE = base_dir + "/google_downloaded_test_set.csv"

# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

new_features.head()

In [ ]:
data, labels, incomplete_sample = prepare_data(new_features)

In [ ]:
from sklearn.metrics import accuracy_score

# Fit to test data

knn_5.fit(data, labels)

prediction = knn_5.predict(data)

knn_5.score(g_data, g_labels)

print('Test Set Accuracy:', accuracy_score(labels, prediction))

### Final Observations

Upon evaluating the trained KNN model on the entire test, the accuracy ended up to be about 100% which for this problem is reasonably acceptable.

## Identifying which type Augmentation improved performance more

### Google Downloaded Fruit Dataset with Noise Augmentation Only

Steps to training:

- Load in the data from the respective directory
- Generate color mean features and save those features as a csv
- Read the save data into a dataframe
- Set the column values from the dataframe
- Display the first 5 rows of feature vector
- Split the feature vector into training and test sets
- Make csv copies of splitted data sets for final testing
- Prepare the data from the KNN model
- Train KNN Model
- Generate new features using PCA (First two PC)
- Retrain KNN Model using PCA features
- Output training results

In [ ]:
path = base_dir + "/noise-augmented-data"

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/noise_augmented_fruit_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

noise_strat_train, noise_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, noise_data, noise_labels, incomplete_1, incomplete_2 = prepare_combined_data(d_new_strat_train, noise_strat_train)

print(incomplete_1)
print(incomplete_2)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_combined_model(g_data, g_labels, noise_data, noise_labels)

In [ ]:
# Perform PCA on dataset

data = append_new_data(g_data, noise_data)
labels = append_new_data(g_labels, noise_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_7 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_7, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights with only the Noise Augmented Data added to the Google Downloaded Dataset, the model prediction results were as follows:

- K = 1 (35% Accuracy)
- K = 3 (35% Accuracy)
- K = 5 (35% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors.

### Google Downloaded Fruit Dataset with Blur Augmentation Only

In [ ]:
path = base_dir + "/blurred-augmented-data"

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/blurred_augmented_fruit_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

blur_strat_train, blur_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, blur_data, blur_labels, incomplete_1, incomplete_2 = prepare_combined_data(d_new_strat_train, blur_strat_train)

print(incomplete_1)
print(incomplete_2)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_combined_model(g_data, g_labels, blur_data, blur_labels)

In [ ]:
# Perform PCA on dataset

data = append_new_data(g_data, blur_data)
labels = append_new_data(g_labels, blur_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_8 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_8, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights with only the Blurred Augmented Data added to the Google Downloaded Dataset, the model prediction results were as follows:

- K = 1 (42% Accuracy)
- K = 3 (35% Accuracy)
- K = 5 (38% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors.

### Google Downloaded Fruit Dataset with Gamma Augmentation Only

In [ ]:
path = base_dir + "/gamma-augmented-data"

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/gamma_augmented_fruit_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

gamma_strat_train, gamma_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, gamma_data, gamma_labels, incomplete_1, incomplete_2 = prepare_combined_data(d_new_strat_train, gamma_strat_train)

print(incomplete_1)
print(incomplete_2)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_combined_model(g_data, g_labels, gamma_data, gamma_labels)

In [ ]:
# Perform PCA on dataset

data = append_new_data(g_data, gamma_data)
labels = append_new_data(g_labels, gamma_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_9 = KNeighborsClassifier(n_neighbors=3)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_9, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights with only the Gamma Augmented Data added to the Google Downloaded Dataset, the model prediction results were as follows:

- K = 1 (35% Accuracy)
- K = 3 (42% Accuracy)
- K = 5 (35% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors.

### Google Downloaded Fruit Dataset with Contrast Augmentation Only

In [ ]:
path = base_dir + "/contrast-augmented-data"

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/contrast_augmented_fruit_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

contrast_strat_train, contrast_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, contrast_data, contrast_labels, incomplete_1, incomplete_2 = prepare_combined_data(d_new_strat_train, contrast_strat_train)

print(incomplete_1)
print(incomplete_2)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_combined_model(g_data, g_labels, contrast_data, contrast_labels)

In [ ]:
# Perform PCA on dataset

data = append_new_data(g_data, contrast_data)
labels = append_new_data(g_labels, contrast_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_10 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_10, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights with only the Contrast Augmented Data added to the Google Downloaded Dataset, the model prediction results were as follows:

- K = 1 (78% Accuracy)
- K = 3 (53% Accuracy)
- K = 5 (53% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors.

### Google Downloaded Fruit Dataset with Horizontal Flipped Augmentation Only

In [ ]:
path = base_dir + "/hflip-augmented-data"

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/hflip_augmented_fruit_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

hflip_strat_train, hflip_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, hflip_data, hflip_labels, incomplete_1, incomplete_2 = prepare_combined_data(d_new_strat_train, hflip_strat_train)

print(incomplete_1)
print(incomplete_2)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_combined_model(g_data, g_labels, hflip_data, hflip_labels)

In [ ]:
# Perform PCA on dataset

data = append_new_data(g_data, hflip_data)
labels = append_new_data(g_labels, hflip_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_11 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_11, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights with only the Horizontal Flipped Augmented Data added to the Google Downloaded Dataset, the model prediction results were as follows:

- K = 1 (88% Accuracy)
- K = 3 (57% Accuracy)
- K = 5 (42% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors.

### Google Downloaded Fruit Dataset with Vertical Flipped Augmentation Only

In [ ]:
path = base_dir + "/vflip-augmented-data"

# Call pre-process raw input function

proc_contents = load_raw_data(path)

In [ ]:
# New data as a csv file

CSV_FILE = base_dir + "/vflip_augmented_fruit_dataset.csv"

save_dataset(proc_contents, CSV_FILE)

In [ ]:
# Download dataset as csv file

# Establish dataset column names

new_features = download_dataset(CSV_FILE, COL_NAMES)

In [ ]:
# View first 5 rows of dataset

new_features.head()

In [ ]:
# Split feature vector into training and testing

vflip_strat_train, vflip_strat_test = splitter(new_features)

In [ ]:
# Prepare the splitted training set for the model

g_data, g_labels, vflip_data, vflip_labels, incomplete_1, incomplete_2 = prepare_combined_data(d_new_strat_train, vflip_strat_train)

print(incomplete_1)
print(incomplete_2)

In [ ]:
# Train/Fit dataset to supervised learning model (KNN)

knn, x_val, y_val, x_train, y_train = train_combined_model(g_data, g_labels, vflip_data, vflip_labels)

In [ ]:
# Perform PCA on dataset

data = append_new_data(g_data, vflip_data)
labels = append_new_data(g_labels, vflip_labels)

train_pca, test_pca = PCA_features(data, labels, x_val, x_train)

In [ ]:
# Use PCA Features to train KNN Model

knn_12 = KNeighborsClassifier(n_neighbors=1)

# Train and evaluate the pca transform data to the original dataset labels

pca_train_model(knn_12, train_pca, test_pca, y_train, y_val)

### Observations

After to creating the dataset of feature vectors using RGB color extraction, transforming the new dataset feature weights into new features weights using principal component analysis and then training a supervised learning model (i.e. KNN) on the new feature weights with only the Vertical Flipped Augmented Data added to the Google Downloaded Dataset, the model prediction results were as follows:

- K = 1 (88% Accuracy)
- K = 3 (57% Accuracy)
- K = 5 (42% Accuracy)
 
Odd numbers for the value of K were used, so as to eliminate any cases of ties when computing the nearest neighbors.